In [1]:
import pandas as pd
import numpy as np
from pandas.core.arrays.sparse import dtype
from model_v1 import MinRiskByHistoricalVar
import matplotlib.pyplot as plt
import ffn
import dask.dataframe
import os
import warnings
import copy
os.chdir('/Users/arthur/Desktop/factor_investment/計算溢酬')

(CVXPY) Apr 28 09:31:49 AM: Encountered unexpected exception importing solver SCS:
ImportError('dlopen(/opt/anaconda3/lib/python3.9/site-packages/_scs_direct.cpython-39-darwin.so, 2): Symbol not found: _aligned_alloc\n  Referenced from: /opt/anaconda3/lib/python3.9/site-packages/scs/.dylibs/libgomp.1.dylib (which was built for Mac OS X 10.15)\n  Expected in: /usr/lib/libSystem.B.dylib\n in /opt/anaconda3/lib/python3.9/site-packages/scs/.dylibs/libgomp.1.dylib')


標準化個因子
考量市值大小
方向變化（正相關 負相關）
缺值等權合成
中性化（https://heatask.com/photograph/69198.html）


In [2]:
# pip install cvxpy

In [3]:
candidatePoolDF=pd.read_excel('./CTBCdata/10個因子原始資料.xlsx','125檔', dtype=str,index_col=0,header=1)
factor_size=pd.read_csv('./CTBCdata/419valuefactor.csv', index_col=0)
factor_size_2=pd.read_excel('./CTBCdata/10個因子原始資料.xlsx','因子值_SIZE', index_col=0)

In [4]:
# factor_size = factor_size.loc[factor_size.index.isin(factor_size_2.index), :]
# factor_size

In [5]:
# factor_size_2

In [6]:
# NoBenchmarkPool = list(factor_size.index)
NoBenchmarkPool=list(candidatePoolDF.index)
candidatePool=['0050']+NoBenchmarkPool
candidatePool=[str(x) for x in candidatePool]
print(len(NoBenchmarkPool))
print(len(candidatePool))

125
126


In [31]:
AllStockReturn=pd.DataFrame()
for stock in candidatePool:
    data=pd.read_csv('./data/price/'+str(stock)+'.csv')

    if stock==candidatePool[0]:
        AllStockReturn=data[['Date','return']]
        AllStockReturn=AllStockReturn.rename(columns={'return':str(stock)})
    else:
        AllStockReturn=AllStockReturn.merge(data[['Date','return']],on='Date',how='left')
        AllStockReturn=AllStockReturn.rename(columns={'return':str(stock)})
AllStockReturn=AllStockReturn[AllStockReturn['Date']>'2010-03-30']
# AllStockReturn=AllStockReturn[AllStockReturn['Date']>'2013-04-30']
dates=AllStockReturn['Date'].values
AllStockReturn=AllStockReturn.reset_index(drop=True)
AllStockReturn

,Date,0050,2330,2317,6505,2412,2454,2882,1301,1303,...,4157,3176,8044,3552,5904,6446,8069,5483,8436,3293
0,2010-03-31,-0.3673,-1.1254,-0.7220,-0.8294,0.0000,0.1818,-2.2140,-0.9901,0.1546,...,NaN,NaN,-1.7751,1.3746,1.7065,NaN,2.3132,0.4957,NaN,2.0833
1,2010-04-01,0.9217,0.9756,1.8182,1.7921,-0.3221,2.7223,0.7547,1.8571,0.7716,...,NaN,NaN,0.1506,-1.0169,-0.6711,NaN,2.6087,0.4932,NaN,-0.6122
2,2010-04-02,0.0000,0.4831,-0.3571,-0.2347,-0.1616,-1.0601,0.1873,-0.1403,-0.3063,...,NaN,NaN,0.9023,-1.7123,-1.1824,NaN,0.0000,1.8405,NaN,0.6160
3,2010-04-06,1.0046,1.6026,2.8674,0.7059,-0.1618,1.7857,0.5607,-0.1404,1.6897,...,NaN,NaN,-0.8942,1.0453,-0.1709,NaN,-3.3898,-0.6024,NaN,1.2245
4,2010-04-07,0.1808,0.1577,0.6969,0.3505,0.1621,-0.5263,0.1859,0.0000,1.8127,...,NaN,NaN,0.1504,-2.0690,0.6849,NaN,1.9298,0.3636,NaN,0.4032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2806,2021-08-20,-0.7967,-1.2522,1.4563,0.2195,-0.4292,-1.6892,-0.1859,-0.9307,-2.5000,...,-2.2727,1.1218,2.4272,-2.5063,-0.7634,-4.5652,5.7534,0.3058,1.1673,-0.9777
2807,2021-08-23,2.7533,2.5362,2.8708,1.6429,-0.4310,4.3528,2.6071,2.2965,2.0757,...,0.5814,-0.9509,1.8957,6.6838,1.7308,1.9362,0.3886,6.0976,0.7692,4.3724
2808,2021-08-24,0.2977,1.0601,1.3953,2.3707,0.0000,-0.2195,1.2704,1.1224,1.5550,...,-1.7341,-2.5600,-1.8605,-3.1325,0.0000,-0.5587,-1.9355,0.0000,0.7634,-0.4298
2809,2021-08-25,0.9647,2.2727,-0.4587,-0.4211,-1.0338,0.2200,0.3584,0.4036,0.3534,...,1.4706,-0.6568,4.7393,2.9851,-0.3781,2.1348,1.3158,9.7701,0.3788,2.3022


In [6]:
def marketValueWeightWithConstrain(factor_size,pool,adjustdate):
   
    newPD=pd.DataFrame()
    for c in factor_size.columns.values:
        newPD[str(str(c).split('T')[0])]=factor_size[c]
        # factor_size=factor_size.rename(columns={c:str(c).split('T')[0]})
    
    chooseFactorDate=newPD.columns.values[np.where(np.array(newPD.columns.values)<=adjustdate)[0][-1]]
    
    NowSizeDF=newPD[chooseFactorDate]
    NowSizeDF=NowSizeDF.reset_index()

    # print(adjustdate,NowSizeDF)
    # print(len(pool))
    NowChoosedPool=NowSizeDF[NowSizeDF['index'].isin(pool)]
    NowChoosedPool=NowChoosedPool.sort_values(by=chooseFactorDate)
    NowChoosedPool=NowChoosedPool.dropna()
    # print(NowChoosedPool)
    Big50Pool=NowChoosedPool.tail(50)
    SumAllBig50Weight=np.sum(Big50Pool[chooseFactorDate].values)
    Big50Pool['initial_weight']=Big50Pool[chooseFactorDate].values/SumAllBig50Weight
    ###Adjust Because of constrain
    AdjustWeight=[]
    ResWeight=[]
    for iw in range(len(Big50Pool['initial_weight'].values)):
        if iw<len(Big50Pool['initial_weight'].values)-1:
            
            Adjust=min(Big50Pool['initial_weight'].values[iw],0.2)
            ResWeight.append(Big50Pool['initial_weight'].values[iw]-Adjust)
            AdjustWeight.append(Adjust)

        else:

            Adjust=min(max(Big50Pool['initial_weight'].values[iw],0.2),0.3)
            ResWeight.append(Big50Pool['initial_weight'].values[iw]-Adjust)
            AdjustWeight.append(Adjust)

    
    Big50Pool['Adjust_weight']=AdjustWeight
    Big50Pool['ResWeight']=ResWeight
    Big50Pool['NoAdjustLabel']=Big50Pool['ResWeight']==0
    SumNoAdjustedWeight=sum(Big50Pool[Big50Pool['ResWeight']==0]['initial_weight'])
    Big50Pool['AdjustRatio']=np.zeros(len(Big50Pool))
    Big50Pool['AdjustRatio']=Big50Pool['initial_weight']/SumNoAdjustedWeight
    Big50Pool['AddWeight']=Big50Pool['NoAdjustLabel']*Big50Pool['AdjustRatio']*sum(ResWeight)
    Big50Pool['FinalWeight']=Big50Pool['Adjust_weight']+Big50Pool['AddWeight']
  
    Big50Pool=Big50Pool[['index','FinalWeight']]
    # print(Big50Pool)
    NewBigPool=pd.DataFrame()
    for i in range(len(Big50Pool)):
      
        NewBigPool[str(Big50Pool['index'].values[i])]=[Big50Pool['FinalWeight'].values[i]]

   
    
    NoWeightStocks=list(set(pool)-set(NewBigPool.columns.values))
    validWeightStocks=list(NewBigPool.columns.values)
    return NewBigPool,NoWeightStocks,validWeightStocks

In [7]:
def marketValueWeightWithoutConstrain(factor_size,pool,adjustdate):
   
    newPD=pd.DataFrame()
    for c in factor_size.columns.values:
        newPD[str(str(c).split('T')[0])]=factor_size[c]
        # factor_size=factor_size.rename(columns={c:str(c).split('T')[0]})
    chooseFactorDate=newPD.columns.values[np.where(np.array(newPD.columns.values)<=adjustdate)[0][-1]]
    
    NowSizeDF=newPD[chooseFactorDate]
    NowSizeDF=NowSizeDF.reset_index()

    # print(adjustdate,NowSizeDF)
    # print(len(pool))
    NowChoosedPool=NowSizeDF[NowSizeDF['index'].isin(pool)]
    NowChoosedPool=NowChoosedPool.sort_values(by=chooseFactorDate)
    NowChoosedPool=NowChoosedPool.dropna()
    # print(NowChoosedPool)
    Big50Pool=NowChoosedPool.tail(50)
    SumAllBig50Weight=np.sum(Big50Pool[chooseFactorDate].values)
   
    
    Big50Pool['FinalWeight']=Big50Pool[chooseFactorDate].values/SumAllBig50Weight
  
    Big50Pool=Big50Pool[['index','FinalWeight']]
    # print(Big50Pool)
    NewBigPool=pd.DataFrame()
    for i in range(len(Big50Pool)):
      
        NewBigPool[str(Big50Pool['index'].values[i])]=[Big50Pool['FinalWeight'].values[i]]
    NoWeightStocks=list(set(pool)-set(NewBigPool.columns.values))
    validWeightStocks=list(NewBigPool.columns.values)
    return NewBigPool,NoWeightStocks,validWeightStocks

In [8]:
AllStockReturn['portfolioReturn']=0
AllStockReturn['portfolioReturnWithCost']=0
AllStockReturn[candidatePool]=AllStockReturn[candidatePool]/100

In [9]:
AllStockWeight=AllStockReturn[AllStockReturn.columns]
for c in NoBenchmarkPool:
    AllStockWeight[str(c)]=np.zeros(len(AllStockWeight))

firstAdjust=False
previousWeight=pd.DataFrame()
weightDF=pd.DataFrame([np.zeros(len(NoBenchmarkPool))],columns=NoBenchmarkPool)

In [11]:
warnings.filterwarnings("ignore")

for idate in range(len(dates)):
    date=dates[idate]
    value=0
    changePool=False
    adjustMonthes=['01','04','07','10']
    
    if idate<len(dates)-1:
        if date.split('-')[1]!=dates[idate+1].split('-')[1]: ##end of month
            if date.split('-')[1] in adjustMonthes:
                changePool=True
                firstAdjust=True
    if firstAdjust==False:
        BeginDate=date
    if changePool:
        previousWeight=weightDF

        ##Without Constrain
        weightDF,NoWeightStocks,validWeightStocks=marketValueWeightWithoutConstrain(factor_size,NoBenchmarkPool,date)
        
        ##With Constrain
        # weightDF,NoWeightStocks,validWeightStocks=marketValueWeightWithConstrain(factor_size,NoBenchmarkPool,date)
       
        weightDF[NoWeightStocks]=0
        weightDF=weightDF.fillna(0)
        previousWeight=previousWeight.fillna(0)
        CostFrame=previousWeight.subtract(weightDF)
        
        SumAllCost=np.nansum(abs(CostFrame.values))
        try:
            EnhanceWeight=MinRiskByHistoricalVar(validWeightStocks,date)
        except:
            print(validWeightStocks,date)
            continue
#         EnhanceWeight=MinRiskByHistoricalVar(validWeightStocks,date)
        EnhanceWeight=EnhanceWeight.fillna(0)
      
        weightDF=pd.concat([weightDF,EnhanceWeight])
       
        Final=weightDF.sum()
        
        minPositive=np.min(Final[Final>0].values)
        Final[Final<0]=minPositive/2
       
        FinalSum=np.sum(Final.values)
        Final=Final/FinalSum
       
        weightDF=pd.DataFrame()
        for i in range(len(validWeightStocks)):
          
            weightDF[validWeightStocks[i]]=[Final[validWeightStocks[i]]]
       
        
    if firstAdjust:
        NowWeightDF=weightDF
        NowWeightDF['Date']=date
        NowReturnDF=AllStockReturn[AllStockReturn['Date']==date]
        
        NowReturnDF=pd.concat([NowReturnDF,NowWeightDF])
        NowReturnDF=NowReturnDF.set_index('Date')
      
        NowReturnDF_temp=NowReturnDF
       
        NowReturnDF_temp=NowReturnDF.dropna(axis=1)
        NowReturnDF_temp=pd.DataFrame(NowReturnDF_temp.apply(np.prod , axis=0))
        
       
        NowReturnDF_temp=NowReturnDF_temp.transpose()
        
        NowReturnDF=pd.concat([NowReturnDF,NowReturnDF_temp])
    
        allStocks=list(NowReturnDF_temp.columns)
       
        # allStocks.remove('0050')
        portReturn=np.nansum(NowReturnDF_temp[allStocks].values[0])

        if changePool==True:
            portReturnCost=portReturn-SumAllCost*0.003
        else:
            portReturnCost=portReturn
        Index=AllStockReturn.index[AllStockReturn['Date'] == date].tolist()
        AllStockReturn.iloc[Index, AllStockReturn.columns.get_loc('portfolioReturn')]=portReturn
        AllStockReturn.iloc[Index, AllStockReturn.columns.get_loc('portfolioReturnWithCost')]=portReturnCost

        # print(AllStockReturn.iloc[Index])

    changePool=False

warnings.filterwarnings("default")

['1301', '9945', '4904', '6121', '5483', '2395', '3481', '5347', '2303', '6239', '2379', '1303', '1102', '3037', '3034', '2002', '6505', '8299', '9921', '1326', '2354', '2105', '1101', '1402', '1227', '2912', '2227', '1504', '2385', '1476', '8044', '5903', '2317', '2492', '2353', '2207', '5904', '2345', '2347', '1216', '9910', '9904', '3702', '2301', '2382', '2377', '2357', '2324', '2356', '3231'] 2010-07-30
['6121', '6239', '1303', '3037', '5347', '9921', '2303', '2379', '2002', '3034', '6505', '1326', '2385', '2344', '2105', '2610', '2912', '1216', '1227', '1402', '1476', '2354', '1101', '8299', '3481', '1504', '8044', '2227', '2618', '2615', '5903', '2409', '2207', '5904', '2353', '2317', '9904', '2347', '2345', '2492', '9910', '3702', '2324', '2301', '2357', '2377', '4938', '2382', '3231', '2356'] 2010-10-29
['3037', '6239', '2379', '3034', '2609', '6505', '2227', '2303', '1326', '9921', '2002', '5347', '2610', '1402', '2105', '8044', '8299', '2912', '1504', '2618', '3481', '2354',

In [12]:
Result=AllStockReturn[['Date','0050','portfolioReturn','portfolioReturnWithCost']]
Result=Result[Result['Date']>BeginDate]
Result=Result.set_index('Date')
Result.index = pd.DatetimeIndex(Result.index)
Result['Rolling_0050']=Result['0050'].rolling(252).apply(lambda x: np.prod(1 + x) - 1)
Result['Rolling_portfolioReturn']=Result['portfolioReturn'].rolling(252).apply(lambda x: np.prod(1 + x) - 1)
Result['Rolling_portfolioReturnWithCost']=Result['portfolioReturnWithCost'].rolling(252).apply(lambda x: np.prod(1 + x) - 1)



Result['0050_Addone']=Result['0050']+1
Result['portfolioReturn_Addone']=Result['portfolioReturn']+1
Result['portfolioReturnWithCost_Addone']=Result['portfolioReturn']+1


# Result['Acc_0050']=Result['0050'].map(lambda x: np.prod(1 + x) - 1)
Result['Acc_0050']=Result['0050_Addone'].cumprod()
# Result['Acc_portfolioReturn']=Result['portfolioReturn'].apply(lambda x: np.prod(1 + x) - 1)
Result['Acc_portfolioReturn']=Result['portfolioReturn_Addone'].cumprod()
Result['Acc_portfolioReturnWithCost']=Result['portfolioReturnWithCost_Addone'].cumprod()



result = Result.calc_stats()
print(f"total return [bench_fund, our_fund,our_fund_cost,]: [{result['Acc_0050'].total_return*100:.2f}%,  {result['Acc_portfolioReturnWithCost'].total_return*100:.2f}%] ")
print(f"annual return [bench_fund, our_fund,our_fund_cost,our_fund_cost_insurance]: [{result['Acc_0050'].cagr*100:.2f}%,  {result['Acc_portfolioReturnWithCost'].cagr*100:.2f}%] ")
print(f"daily sharpe [bench_fund, our_fund,our_fund_cost,our_fund_cost_insurance]: [{result['Acc_0050'].daily_sharpe:.2f},  {result['Acc_portfolioReturnWithCost'].daily_sharpe:.2f}] ")
print(f"max drawdown [bench_fund, our_fund,our_fund_cost,our_fund_cost_insurance]: [{result['Acc_0050'].max_drawdown*100:.2f}%,  {result['Acc_portfolioReturnWithCost'].max_drawdown*100:.2f}%] ")
print(f"daily_sortino [bench_fund, our_fund,our_fund_cost,our_fund_cost_insurance]: [{result['Acc_0050'].daily_sortino*100:.2f}%,  {result['Acc_portfolioReturnWithCost'].daily_sortino*100:.2f}%] ")
print(f"daily_vol [bench_fund, our_fund,our_fund_cost,our_fund_cost_insurance]: [{result['Acc_0050'].daily_vol*100:.2f}%,  {result['Acc_portfolioReturnWithCost'].daily_vol*100:.2f}%] ")
print(f"daily_skew [bench_fund, our_fund,our_fund_cost,our_fund_cost_insurance]: [{result['Acc_0050'].daily_skew*100:.2f}%,  {result['Acc_portfolioReturnWithCost'].daily_skew*100:.2f}%] ")
print(f"daily_kurt [bench_fund, our_fund,our_fund_cost,our_fund_cost_insurance]: [{result['Acc_0050'].daily_kurt*100:.2f}%,  {result['Acc_portfolioReturnWithCost'].daily_kurt*100:.2f}%] ")
print(f"calmar [bench_fund, our_fund,our_fund_cost,our_fund_cost_insurance]: [{result['Acc_0050'].calmar*100:.2f}%,  {result['Acc_portfolioReturnWithCost'].calmar*100:.2f}%] ")



plt.subplot(2,1,1)
plt.plot(Result['Rolling_0050'].values, color='red')
plt.plot(Result['Rolling_portfolioReturn'].values, color='yellow')
plt.plot(Result['Rolling_portfolioReturnWithCost'].values, color='blue')
plt.subplot(2,1,2)
plt.plot(Result['Acc_0050'].values, color='red')
plt.plot(Result['Acc_portfolioReturn'].values, color='yellow')
plt.plot(Result['Acc_portfolioReturnWithCost'].values, color='blue')
plt.savefig('performance_2')
plt.close

# Result['Rolling_0050']=pd.rolling_apply(AllStockReturn[['0050']], 252, lambda x: np.prod(1 + x) - 1)
# Result['portfolioReturn']=pd.rolling_apply(AllStockReturn[['portfolioReturn']], 252, lambda x: np.prod(1 + x) - 1)
print(Result)

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/anaconda3/lib/python3.9/site-packages/ffn/core.py:1376: RuntimeWarning: invalid value encountered in double_scalars
  return (prices.iloc[-1] / prices.iloc[0]) ** (1 / year_frac(start, end)) - 1
/opt/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/opt/anaconda3/lib/python3.9/site-packages/ffn/core.py:405: RuntimeWarning: divide by zero encountered in double_scalars
  if mp[i] / mp[i - 11] > 1:
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: divide by zero encountered

total return [bench_fund, our_fund,our_fund_cost,]: [225.34%,  531.90%] 
annual return [bench_fund, our_fund,our_fund_cost,our_fund_cost_insurance]: [12.44%,  20.10%] 
daily sharpe [bench_fund, our_fund,our_fund_cost,our_fund_cost_insurance]: [0.82,  1.13] 
max drawdown [bench_fund, our_fund,our_fund_cost,our_fund_cost_insurance]: [-28.24%,  -35.89%] 
daily_sortino [bench_fund, our_fund,our_fund_cost,our_fund_cost_insurance]: [134.66%,  174.63%] 
daily_vol [bench_fund, our_fund,our_fund_cost,our_fund_cost_insurance]: [16.29%,  18.08%] 
daily_skew [bench_fund, our_fund,our_fund_cost,our_fund_cost_insurance]: [-9.47%,  -77.63%] 
daily_kurt [bench_fund, our_fund,our_fund_cost,our_fund_cost_insurance]: [553.96%,  506.25%] 
calmar [bench_fund, our_fund,our_fund_cost,our_fund_cost_insurance]: [44.05%,  56.01%] 
                0050  portfolioReturn  portfolioReturnWithCost  Rolling_0050  \
Date                                                                           
2010-07-30 -0.003697   